In [1]:
from aeroTesting import AeroRepeatability, Probe

In [ ]:
# user sequence:
golden_probe_path = input('Path to golden probe data: ')
uut_probe_path = input('Path to uut probe data: ')
AeroRepeatability(golden_probe_path, uut_probe_path)

In [2]:
golden_probe_path = '/home/stoker/aero_analysis/A0251-50kn'
uut_probe_path = '/home/stoker/aero_analysis/SN003-50kn'

In [ ]:
uut_probe.data_path

In [3]:
AeroRepeatability(golden_probe_path, uut_probe_path)

Known headers:
aerosonic_default
wrong
test1
Choose an existing header or type NO:aerosonic_default
{'1': 'time', '9': 'tempF', '7': 'uut_static_pressure2', '10': 'angle', '3': 'tunnel_ref_static_pressure', '0': 'date', '6': 'uut_static_pressure1', '2': 'tunnel_ref_total_pressure', '5': 'uut_total_pressure'}
{'1': 'time', '9': 'tempF', '7': 'uut_static_pressure2', '10': 'angle', '3': 'tunnel_ref_static_pressure', '0': 'date', '6': 'uut_static_pressure1', '2': 'tunnel_ref_total_pressure', '5': 'uut_total_pressure'}


/home/stoker/aero_analysis/aeroTesting.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['angle'] = df['angle'].apply(self.normalize_angle)


In [6]:
golden_probe = Probe(golden_probe_path)


Known headers:
wrong
test1
aerosonic_default
Choose an existing header or type NO:aerosonic_default
{'0': 'date', '1': 'time', '6': 'uut_static_pressure1', '2': 'tunnel_ref_total_pressure', '5': 'uut_total_pressure', '7': 'uut_static_pressure2', '10': 'angle', '3': 'tunnel_ref_static_pressure', '9': 'tempF'}


/home/stoker/aero_analysis/aeroTesting.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['angle'] = df['angle'].apply(self.normalize_angle)


In [7]:
uut_probe = Probe(uut_probe_path)

Known headers:
wrong
test1
aerosonic_default
Choose an existing header or type NO:aerosonic_default
{'0': 'date', '1': 'time', '6': 'uut_static_pressure1', '2': 'tunnel_ref_total_pressure', '5': 'uut_total_pressure', '7': 'uut_static_pressure2', '10': 'angle', '3': 'tunnel_ref_static_pressure', '9': 'tempF'}


/home/stoker/aero_analysis/aeroTesting.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['angle'] = df['angle'].apply(self.normalize_angle)


In [ ]:
uut_probe.cp

In [14]:
#import seaborn
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from datetime import date
import ntpath
#def compare_cp_plot(self, col, origin_bounds = 0.005):
#    #TODO
#    return
origin_bounds = 0.005

In [15]:
col = 'cp_total_pressure'
filename = uut_probe.data_path
if 'total' in col:
    generated_title = 'Total Pressure'
    filename = filename + '-total-'
elif 'static' in col and '1' in col:
    generated_title = 'Static Pressure 1'
    filename = filename + '-static1-'
elif 'static' in col and '2' in col:
    generated_title = 'Static Pressure 2'
    filename = filename + '-static2-'
else:
    generated_title = ''
filename = filename + '-' + str(date.today()) + '.png'
# check if passed or failed
difference = uut_probe.cp[col]-golden_probe.cp[col]
difference.drop(labels='Drift Data', inplace=True)
pos_mask = difference.index.to_series() >= 0
neg_mask = difference.index.to_series() < 0
upper_bound = pd.concat([origin_bounds + difference.index.to_series()[pos_mask]*0.0005, origin_bounds - difference.index.to_series()[neg_mask]*0.0005])
upper_bound.sort_index()
lower_bound = pd.concat([-origin_bounds - difference.index.to_series()[pos_mask]*0.0005, -origin_bounds + difference.index.to_series()[neg_mask]*0.0005])



if any(upper_bound - difference < 0):
    passFail = 'FAIL'
elif any(difference - lower_bound < 0):
    passFail = 'FAIL'
else:
    passFail = 'PASS'

neg_mask = difference.index.to_series() <= 0
plt.figure(figsize=(8, 6), dpi=80)
plt.scatter(difference.index, difference, color='blue')

#plt.plot(upper_bound.index, upper_bound, linestyle='dashed', color='red')
plt.plot(difference.index[pos_mask], origin_bounds + difference.index.to_series()[pos_mask]*0.0005, linestyle='dashed', color='red')
plt.plot(difference.index[pos_mask], -origin_bounds - difference.index.to_series()[pos_mask]*0.0005, linestyle='dashed', color='red')
plt.plot(difference.index[neg_mask], origin_bounds - difference.index.to_series()[neg_mask]*0.0005, linestyle='dashed', color='red')
plt.plot(difference.index[neg_mask], -origin_bounds + difference.index.to_series()[neg_mask]*0.0005, linestyle='dashed', color='red')
plt.xlabel('AOA (deg)')
plt.ylabel('$\Delta$P / $q_c$')
plt.title(generated_title + '\n ' + passFail)

#plt.show()
plt.savefig(filename)





In [23]:
pf_bool = (upper_bound - difference < 0) | (difference - lower_bound < 0)# true if out of bounds
pf = pd.DataFrame(['P']*len(pf_bool.index), index = pf_bool.index)
pf[pf_bool] = 'F'
pf

,0
angle,
-20,F
-18,F
-16,F
-14,F
-12,F
-10,F
-8,F
-6,F
-4,F


In [ ]:
path = os.path.join(golden_probe.data_path.split('/')[:-1])
path

In [ ]:
os.path.pathsep#(golden_probe.data_path)